In [ ]:
matplotlib notebook

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import mne

In [ ]:
# getting info without loading data
# reading locally:
raw = mne.io.read_raw_edf('../a1d36553_8.edf',preload = False)
# reading from cylon:
# raw = mne.io.read_raw_edf('/data1/edf/a1d36553_8/a1d36553_8.edf',preload = False)

In [ ]:
raw.plot(block=True, lowpass=40)

In [ ]:
# Checking out the file info
info = raw.info
print(info)

In [ ]:
# channel names
info['ch_names']

Now let's convert the times to datetime.

In [ ]:
import datetime

In [ ]:
basetime_posix = info['meas_date']

In [ ]:
# Check out the date
datetime.datetime.fromtimestamp(
        basetime_posix
    ).strftime('%Y-%m-%d %H:%M:%S')

In [ ]:
channel1 = raw.get_data(picks=1, start=0, stop=400000, reject_by_annotation=None, return_times=True)

In [ ]:
channel1[1].ravel().shape

In [ ]:
import pandas as pd

datetimes = pd.to_datetime(channel1[1].ravel()+basetime_posix, unit = 's')

In [ ]:
plt.figure()
plt.plot(datetimes, channel1[0].ravel())
plt.plot()
plt.title('The first channel')

Seems there is some crazy stuff happening at the beginning of the session. Let's skip the beginning.

In [ ]:
plt.figure()
plt.plot(datetimes[200000:], channel1[0].ravel()[200000:])
plt.plot()
plt.title('The first channel')

### Calculating Power Spectrum Density

There are many channels in the dataset, but we are only interested in the eeg channels. We can pick those using a the `pick_types` function:

In [ ]:
picks = mne.pick_types(raw.info, meg=False, eeg=True, eog=False,
                       stim=False, exclude='bads')

print('There are a total of '+str(len(picks))+' channels.')

In [ ]:
# Let's just look at the first few channels for demonstration purposes
picks = picks[10:30] # some of the channels still are bad (check this thoroughly)

We will show how to obtain the power spectral density for a segment of the time series.

In [ ]:
fmin, fmax = 0.1, 160  # need to ask what a good range is (I think it is in Nancy's paper)
n_fft = 2048  # the FFT size (n_fft). Ideally a power of 2

plt.figure()
ax = plt.axes()
raw.plot_psd(tmin=2000, tmax=10000, fmin=fmin, fmax=fmax, n_fft=n_fft,
             n_jobs=1, proj=False, ax=ax, color=(0, 0, 1),  picks=picks,
             show=False, average=True)
plt.xlabel('Frequency')
plt.ylabel('PSD')


The power spectrum above represents features for the time series between time points 2000-10000. To relate to the emotions features we want to calculate the power spectrum at each point for a small window. For that you can use another function which calculates psd per window: `mne.time_frequency.psd_welch`.

In [ ]:
help(mne.time_frequency.psd_welch)